### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Variables created to determine length of dataframe, where unique is used to ensure no repeat ID numbers.
schools = len(school_data_complete['School ID'].unique())
students = len(school_data_complete['Student ID'].unique())
# Variable calculates sum of budget column in dataframe
budget = school_data['budget'].sum()
# Variables calculate average of math and reading scores
avg_math = school_data_complete['math_score'].mean()
avg_read = school_data_complete['reading_score'].mean()
# Creates new string including only students whose math scores were >= 70
passing_math = school_data_complete[(school_data_complete['math_score'] >= 70)]
# Determines total numbers of students who passed math and converts to percentage
passing_math_percent = (len(passing_math) / students) * 100
# Creates new string including only students whose reading scores were >= 70
passing_read = school_data_complete[(school_data_complete['reading_score'] >= 70)]
# Determines total numbers of students who passed reading and converts to percentage
passing_read_percent = (len(passing_read) / students) * 100
# Takes passing_math string and selects for students who also passed reading
passing_both_df = passing_math[(passing_math['reading_score'] >= 70)]
# Determines length of string for both passing and converts to percentage
passing_both_percent = (len(passing_both_df) / students) * 100
# Creates new dataframe to print results with proper formatting
results_df = pd.DataFrame({"Total Schools": [schools], 
                           "Total Students": ["{:,}".format(students)], 
                           "Total Budget": ["${:,.2f}".format(budget)], 
                           "Avg Math Score": ["{:.1f}".format(avg_math)],
                           "Avg Reading Score": ["{:.1f}".format(avg_read)],
                           "% Pass Math": ["{:.1f}".format(passing_math_percent)],
                           "% Pass Read": ["{:.1f}".format(passing_read_percent)],
                            "% Pass Overall": ["{:.1f}".format(passing_both_percent)],
                            })
results_df

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Pass Math,% Pass Read,% Pass Overall
0,15,"39,170","$24,649,428.00",79.0,81.9,75.0,85.8,65.2


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# Cleaning the dataframe of un-needed columns
school_clean = school_data_complete[['school_name', 'type', 'math_score', 'reading_score', 'size', 'budget']]
# Creating an index for school name and type
school_index = school_clean.set_index(['school_name', 'type'])
# Grouping by the index. ***This was the only way I could figure out how to match each school with its respective type
school_group = school_index.groupby(['school_name', 'type'])
# Calculating the mean across the dataframe.  ***Again, only way I could get school type to match school.
# I would be interested in seeing an alternate solution for this.
grouped_mean = school_group.mean()
# Taking the school type index out and replacing it as a column in the dataframe. ***Related to hack described above.
grouped_mean = grouped_mean.reset_index(level = 'type')
# Creating a clean dataframe with only the columns I will use
grouped_clean = grouped_mean.loc[:, ['type', 'reading_score', 'math_score', 'size', 'budget']]
# Creating a new column in the dataframe comprised of a calculation for per student budget
grouped_clean['Per Student Budget'] = grouped_clean['budget'] / grouped_clean['size']
# Dataframe variable to display only students with a math score >= 70
passing_math = school_data_complete.loc[school_data_complete['math_score'] >= 70]
# variable to group by passing match score and counts the total number of passing math scores
perschool = passing_math.groupby(['school_name']).count()
# clean varialbe to contain only the counts for passing math scores
perschool1 = perschool[['math_score']]
# New column inserted into grouped_clean dataframe with results of calculation for % who passed math
grouped_clean['% Pass Math'] = (perschool1['math_score'] / grouped_clean['size']) *100
# Repeats the above steps starting at "passing math" from above, except for reading
passing_reading = school_data_complete.loc[school_data_complete['reading_score'] >= 70]

perschool_read = passing_reading.groupby(['school_name']).count()

perschool2 = perschool_read[['reading_score']]

grouped_clean['% Pass Read'] = (perschool2['reading_score'] / grouped_clean['size']) *100
# Repeats the above steps starting at "passing math" from above, except for overall passing
overall_passing_df = (school_data_complete.loc[(school_data_complete['math_score']>=70) & (school_data_complete['reading_score']>=70)])

perschool_overall = overall_passing_df.groupby(['school_name']).count()

perschool3 = perschool_overall[['reading_score']]

grouped_clean['% Pass Overall'] = (perschool3['reading_score'] / grouped_clean['size']) *100
# Renaming columns to read better
grouped_rename = grouped_clean.rename(columns={'type': 'Type', 'size': 'Total Students', 'reading_score': 'Avg Reading Score', 'math_score': 'Avg Math Score', 'budget': 'Total School Budget'})
# Establish formatting selected columns for currency
format_mapping = {'Total School Budget' : '${:,.0f}', 'Per Student Budget' : '${:,.0f}'}
# Applying formatting
grouped_rename_formatted = grouped_rename.style.format(format_mapping)
# Prints results
grouped_rename_formatted


,Type,Avg Reading Score,Avg Math Score,Total Students,Total School Budget,Per Student Budget,% Pass Math,% Pass Read,% Pass Overall
school_name,,,,,,,,,
Bailey High School,District,81.033963,77.048432,4976.000000,"$3,124,928",$628,66.680064,81.933280,54.642283
Cabrera High School,Charter,83.975780,83.061895,1858.000000,"$1,081,356",$582,94.133477,97.039828,91.334769
Figueroa High School,District,81.158020,76.711767,2949.000000,"$1,884,411",$639,65.988471,80.739234,53.204476
Ford High School,District,80.746258,77.102592,2739.000000,"$1,763,916",$644,68.309602,79.299014,54.289887
Griffin High School,Charter,83.816757,83.351499,1468.000000,"$917,500",$625,93.392371,97.138965,90.599455
Hernandez High School,District,80.934412,77.289752,4635.000000,"$3,022,020",$652,66.752967,80.862999,53.527508
Holden High School,Charter,83.814988,83.803279,427.000000,"$248,087",$581,92.505855,96.252927,89.227166
Huang High School,District,81.182722,76.629414,2917.000000,"$1,910,635",$655,65.683922,81.316421,53.513884
Johnson High School,District,80.966394,77.072464,4761.000000,"$3,094,650",$650,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
# Uses the above dataframe to sort for % passing overall, with ascending specified
top_sort = grouped_rename.sort_values(by=['% Pass Overall'], ascending=False)
# Selects for top 5 schools for all data in the dataframe and also all columns
top_five = top_sort.iloc[0:5, :]
top_five

,Type,Avg Reading Score,Avg Math Score,Total Students,Total School Budget,Per Student Budget,% Pass Math,% Pass Read,% Pass Overall
school_name,,,,,,,,,
Cabrera High School,Charter,83.975780,83.061895,1858.0,1081356.0,582.0,94.133477,97.039828,91.334769
Thomas High School,Charter,83.848930,83.418349,1635.0,1043130.0,638.0,93.272171,97.308869,90.948012
Griffin High School,Charter,83.816757,83.351499,1468.0,917500.0,625.0,93.392371,97.138965,90.599455
Wilson High School,Charter,83.989488,83.274201,2283.0,1319574.0,578.0,93.867718,96.539641,90.582567
Pena High School,Charter,84.044699,83.839917,962.0,585858.0,609.0,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
# See notes from above cell.  Same procedure here.
bottom_sort = grouped_rename.sort_values(by=["% Pass Overall"], ascending=True)
bottom_five = bottom_sort.iloc[0:5, :]
bottom_five

,Type,Avg Reading Score,Avg Math Score,Total Students,Total School Budget,Per Student Budget,% Pass Math,% Pass Read,% Pass Overall
school_name,,,,,,,,,
Rodriguez High School,District,80.744686,76.842711,3999.0,2547363.0,637.0,66.366592,80.220055,52.988247
Figueroa High School,District,81.158020,76.711767,2949.0,1884411.0,639.0,65.988471,80.739234,53.204476
Huang High School,District,81.182722,76.629414,2917.0,1910635.0,655.0,65.683922,81.316421,53.513884
Hernandez High School,District,80.934412,77.289752,4635.0,3022020.0,652.0,66.752967,80.862999,53.527508
Johnson High School,District,80.966394,77.072464,4761.0,3094650.0,650.0,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# New df selecting only for columns with applicable information
grade_clean = school_data_complete[['school_name', 'grade', 'math_score']]
# New variable as a series that selects for 9th graders
nineths = grade_clean.loc[grade_clean['grade'] == '9th']
# New variable as a series that groups by school name
nineths_group = nineths.groupby(['school_name'])
# New variable as a series that calculates the mean of values in previous series.
nineths_group_avg = nineths_group.mean()

# Rinse and repeat.  See above.
tenths = grade_clean.loc[grade_clean['grade'] == '10th']

tenths_group = tenths.groupby(['school_name'])

tenths_group_avg = tenths_group.mean()

# Rinse and repeat.  See above.
eleventh = grade_clean.loc[grade_clean['grade'] == '11th']

eleventh_group = eleventh.groupby(['school_name'])

eleventh_group_avg = eleventh_group.mean()

# Rinse and repeat.  See above.
twelves = grade_clean.loc[grade_clean['grade'] == '12th']

twelves_group = twelves.groupby(['school_name'])

twelves_group_avg = twelves_group.mean()

# New df using merge function to merge 9th and 10th grade
merged_math1 = pd.merge(nineths_group_avg, tenths_group_avg, how="outer", on=["school_name"])
# Renaming columns in newly merged df for readibility
merged_math1 = merged_math1.rename(columns={'math_score_x': '9th', 'math_score_y': '10th'})
# New df merging df comprised of 9th and 10th, plus 11th grade
merged_math_next = pd.merge(merged_math1, eleventh_group_avg, how='outer', on=['school_name'])
# Another merge to add 12th grade
merged_math_final = pd.merge(merged_math_next, twelves_group_avg, how='outer', on=['school_name'])
# Renaming columns for readibility
merged_math_final = merged_math_final.rename(columns={'9th': '9th', '10th': '10th','math_score_x': '11th', 'math_score_y': '12th'})
# Prints results
merged_math_final

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# Same exact procedure as above, except reading scores replace math scores

grade_clean = school_data_complete[['school_name', 'grade', 'reading_score']]

nineths = grade_clean.loc[grade_clean['grade'] == '9th']

nineths_group = nineths.groupby(['school_name'])

nineths_group_avg = nineths_group.mean()


tenths = grade_clean.loc[grade_clean['grade'] == '10th']

tenths_group = tenths.groupby(['school_name'])

tenths_group_avg = tenths_group.mean()


eleventh = grade_clean.loc[grade_clean['grade'] == '11th']

eleventh_group = eleventh.groupby(['school_name'])

eleventh_group_avg = eleventh_group.mean()


twelves = grade_clean.loc[grade_clean['grade'] == '12th']

twelves_group = twelves.groupby(['school_name'])

twelves_group_avg = twelves_group.mean()


merged_reading1 = pd.merge(nineths_group_avg, tenths_group_avg, how="outer", on=["school_name"])

merged_reading1 = merged_reading1.rename(columns={'reading_score_x': '9th', 'reading_score_y': '10th'})

merged_reading_next = pd.merge(merged_reading1, eleventh_group_avg, how='outer', on=['school_name'])

merged_reading_final = pd.merge(merged_reading_next, twelves_group_avg, how='outer', on=['school_name'])

merged_reading_final = merged_reading_final.rename(columns={'9th': '9th', '10th': '10th','reading_score_x': '11th', 'reading_score_y': '12th'})

merged_reading_final

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
# New df selecting only for applicable columns 
school_spending = grouped_rename[['Per Student Budget', 'Avg Reading Score', 'Avg Math Score', '% Pass Math', '% Pass Read', '% Pass Overall']]
# Removing school name index
school_spending = school_spending.reset_index(drop=True)
# Establishing bin values
bin_values = [0, 584.9, 629.9, 644.9, 680]
# Establishing bin names
bin_names = ['<$585', '$585 - 630', '$630 - 645', '$645 - 680']
# New column in df named "Spending Range per Student" that determines bin name based on value in "Per Student Budget"
school_spending["Spending Range per Student"] = pd.cut(school_spending["Per Student Budget"], bin_values, labels=bin_names, include_lowest=True)
# Grouping by newly generated column from above step
school_grouped = school_spending.groupby('Spending Range per Student')
# Calculating average of values in the df
school_grouped_avg = school_grouped[['Avg Math Score', 'Avg Reading Score', '% Pass Math', '% Pass Read', '% Pass Overall']].mean()
# Prints results
school_grouped_avg

,Avg Math Score,Avg Reading Score,% Pass Math,% Pass Read,% Pass Overall
Spending Range per Student,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585 - 630,81.899826,83.155286,87.133538,92.718205,81.418596
$630 - 645,78.518855,81.624473,73.484209,84.391793,62.857656
$645 - 680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Same procedure as above, with school size replacing school spending, however only 3 bins created this time
school_size = grouped_rename[['Total Students', 'Avg Reading Score', 'Avg Math Score', '% Pass Math', '% Pass Read', '% Pass Overall']]

school_size = school_size.reset_index(drop=True)

bin_values = [0, 999.9, 1999.9, 5000]

bin_names = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

school_size["School Size"] = pd.cut(school_size["Total Students"], bin_values, labels=bin_names, include_lowest=True)

size_grouped = school_size.groupby('School Size')

size_grouped_avg = size_grouped[['Avg Math Score', 'Avg Reading Score', '% Pass Math', '% Pass Read', '% Pass Overall']].mean()

size_grouped_avg

,Avg Math Score,Avg Reading Score,% Pass Math,% Pass Read,% Pass Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
# Again, same as above, however since bin values equal bin names, the pd.cut function wasn't required.
school_type = grouped_rename[['Type', 'Avg Reading Score', 'Avg Math Score', '% Pass Math', '% Pass Read', '% Pass Overall']]

school_type = school_type.reset_index(drop=True)

type_grouped = school_type.groupby('Type')

type_grouped_avg = type_grouped[['Avg Math Score', 'Avg Reading Score', '% Pass Math', '% Pass Read', '% Pass Overall']].mean()

type_grouped_avg

,Avg Math Score,Avg Reading Score,% Pass Math,% Pass Read,% Pass Overall
Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [11]:
#Analysis:
# 1. Large district schools have the lowest overall passing percentage even though they have high spend per student.
# 2. Charter schools are more financially efficient at producing students who pass overall, likely because they are small to medium in size
# suggesting a better student/teacher experience.